In [2]:
import pandas as pd
print("Executed.........")

Executed.........


In [3]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup

url = "https://www.jiomart.com/c/groceries/staples/13"

# Initialize the webdriver
driver = webdriver.Chrome()  # You need to have chromedriver installed and in your PATH

# Open the URL
driver.get(url)

# Scroll to the bottom of the page to load more items
scroll_pause_time = 7 # Adjust this value as needed
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Once all items are loaded, extract the HTML content
html_content = driver.page_source

# Close the webdriver
driver.quit()

# Use BeautifulSoup to parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all items using a refined selector
results = soup.find_all('div', {'class': 'plp-card-details-container'})

# Now you can proceed with processing the results
for result in results:
    # Process each item as needed
    product_name = result.find('div', {'class': 'plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80'})
    
    product_price = result.find('div', {'class': 'jm-heading-xxs jm-mb-xxs'})
    product_discount = result.find('span', {'class': 'jm-badge'})

    print("Product Name:", product_name)
    print("Price:", product_price)
    print("Discount:", product_discount)
    print()  # Print an empty line for better readability


Product Name: <div class="plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80"> Bharat Chana Dal 1 Kg </div>
Price: None
Discount: None

Product Name: <div class="plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80"> Just Pressed Cold Pressed Safflower Oil | 100% Pure Safflower Kusum Saffola Oil | Unrefined &amp; Zero Additives | Healthy Heart Oil | Saffola Hero Ingredient | Kachi Ghani Oil | Wood Cold Pressed Oil | Safflower Oil 5000 ML </div>
Price: None
Discount: <span class="jm-badge"> 34% OFF </span>

Product Name: <div class="plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80"> Super Sarvottam Physically Refined Rice Bran Oil 1 L </div>
Price: None
Discount: <span class="jm-badge"> 51% OFF </span>

Product Name: <div class="plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80"> Saffola Active Pro Weight Watchers Rice Bran Based Blended Oil 1 L </div>
Price: None
Discount: <span class="jm-badge"> 33% OFF </span>

Product Name

In [4]:
len(results)

322

In [6]:
# Puting Everything in loop

product_name = []
product_price = []
product_discount = []
for i in range(len(results)):
    badge_element = results[i].find('span', {'class': 'jm-badge'})
    product_name.append(results[i].find('div',{'class':'plp-card-details-name line-clamp jm-body-xs jm-fc-primary-grey-80'}).get_text().strip())
    product_price.append(results[i].find('span',{'class':'jm-heading-xxs jm-mb-xxs'}).get_text().strip())
    if badge_element is not None:
        text_inside_badge = badge_element.get_text().strip()
        if text_inside_badge:  # Check if the text is not an empty string
            product_discount.append(text_inside_badge)
        else:
            # Handle the case when the text is empty, e.g., set a default value
            product_discount.append("None")  # Replace "No Discount" with your desired default value
    else:
        # Handle the case when the element is not found, e.g., set a default value
        product_discount.append("None")
print("Executed.....................")

Executed.....................


In [7]:
data = {'Product Name':product_name,'Price':product_price,'Discount':product_discount}
df = pd.DataFrame(data)
print("executed......")

executed......


In [8]:
df['Discount'] = df['Discount'].replace('None','0% OFF')
print("executed....")

executed....


In [9]:
# Price C
df['Price'] = df['Price'].str[1:]
print("executed....")

executed....


In [10]:
df['Price'] = df['Price'].str.replace(',', '').astype(float)
print("executed....")

executed....


In [11]:
# discount data from 0% off to 0
df['Discount'] = df['Discount'].str[0:-5]
print("executed....")

executed....


In [12]:
df

,Product Name,Price,Discount
0,Bharat Chana Dal 1 Kg,60.0,0
1,Just Pressed Cold Pressed Safflower Oil | 100%...,1699.0,34
2,Super Sarvottam Physically Refined Rice Bran O...,97.0,51
3,Saffola Active Pro Weight Watchers Rice Bran B...,106.0,33
4,Tata Iodised Salt 1 kg,24.0,14
...,...,...,...
317,Good Life Big Crystal Diamond Sugar 200 g,20.0,9
318,The Oil Factory 5.25L Extra Light Olive Oil | ...,1700.0,65
319,Catch Turmeric Powder 200 g,48.0,33
320,Future life Oats Atta | Whole Grains Oats | Wi...,185.0,15


In [13]:
# converting discount column to FLOAT
df['Discount'] = df['Discount'].astype(float)
print("executed......")

executed......


In [14]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce')
print("executed..........")

executed..........


In [15]:
df['Original Price'] = df['Price']/ (1-df['Discount']/100)
print('executed..........')

executed..........


In [16]:
df['Original Price'] = round(df['Original Price'])
print('executed ...... ...... ..')

executed ...... ...... ..


In [17]:
df['Original Price'].max()

4857.0

In [20]:
df.to_csv('jiomart/Jiomart_staples(16-02-2024).csv',index=False)

### Snacks